In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("catalog", "hive_metastore")
dbutils.widgets.text("schema_name", "delta_nip")
dbutils.widgets.text("root_path", "/user/hive/warehouse/delta_nip.db/")
dbutils.widgets.text("table_name", "ref_marketing_plan")
dbutils.widgets.text("file_name", "ref_marketing_plan_exported_data_04052025.csv")

In [0]:

catalog = dbutils.widgets.get("catalog")
schema_name = dbutils.widgets.get("schema_name")
root_path = dbutils.widgets.get("root_path")
table_name = dbutils.widgets.get("table_name")
file_name = dbutils.widgets.get("file_name")
table_full_path = f"{root_path}{table_name}"

# Step 1; Read schema from existing Delta table
existing_table_schema = spark.read.format("delta").load(table_full_path).schema

# Step 2: Load data from CSV and enforce schema from existing table to the DataFrame
mnt_path = f"/mnt/load/{file_name}"
df = spark.read.csv(mnt_path, header=True, schema=existing_table_schema)

# Options include:
# append: Append contents of this DataFrame to existing data.
# overwrite: Overwrite existing data.
# error or errorifexists: Throw an exception if data already exists.
# ignore: Silently ignore this operation if data already exists.

# Step 3: Write DataFrame to Delta table
df.write.format("delta").mode("append").saveAsTable(table_full_path)